In [81]:
from typing import overload, Callable, Union, Optional, List

from sympy import latex
from sympy.core._print_helpers import Printable
from IPython.display import Latex

# Should be set to 'False' if Google Colab is used.
VS_CODE: bool = True

@overload
def display_with_text(text: Optional[str] = None) -> None:
    ...

@overload
def display_with_text(text: Optional[str] = None, expression: Optional[Union[str, Printable]] = None) -> None:
    ...

@overload
def display_with_text(text: Optional[str] = None, *expressions: Union[str, Printable]) -> None:
    ...

def display_with_text(*args, **kwargs) -> None:
    text: Optional[str] = kwargs.get("text", None)
    expression: Optional[Union[str, Printable]] = kwargs.get("expression", None)

    if text is None and args:
        text = args[0]
    
    expressions: List[Union[str, Printable]] = \
        [expr for expr in args[1:] if isinstance(expr, (str, Printable))] \
        if len(args) > 1 else []

    if expression is not None:
        expressions.append(expression)

    if not expressions:
        if text is not None:
            text = "$ \\text{" + text + "} $" if VS_CODE else "\\text{" + text + '}'
            display(Latex(text))
        return
    
    verifier: Callable[[Union[str, Printable]], str] = \
        lambda expr: expr if isinstance(expr, str) else str(latex(expr))
    final_expression: str = r"\\".join(map(verifier, expressions))
    
    if VS_CODE:
        final_expression = "$ " + final_expression + " $"

    if text is not None:
        final_expression = \
            ("$ \\text{" + text + "} $ " if VS_CODE else "\\text{" + text + "} ") \
            + final_expression

    display(Latex(final_expression))


In [82]:
import sympy
from sympy import S, Symbol, symbols, simplify, nonlinsolve, Matrix, diff, factor, Expr

## Занятие 15
## Математический анализ
## Экстремум функции нескольких переменных 
### Задание 1
Найти точки экстремума:

1) $z = x^2 + xy + y^2 - 2x - y$

2) $z = x^3y^2(6 - x - y)$, $(x, y > 0)$

3) $z = \frac{8}{x} + \frac{x}{y} + y$

4) $z = \frac{1 + x - y}{\sqrt{1 + x^2 + y^2}}$
###### Указание.
Вначале найти стационарные точки. Составить определитель из вторых производных в произвольной точке. Подставляя найденные (вещественные) стационарные точки, решить вопрос о наличии экстремума.

In [83]:
from sympy.calculus.util import continuous_domain

def Delta(f: Expr, x: Expr, y: Expr):
    return Matrix([
        [simplify(f.diff(x, 2)), simplify(f.diff(x, y))],
        [simplify(f.diff(y, x)), simplify(f.diff(y, 2))],
    ]).det()

def find_extr_points(f: Expr, x: Expr, y: Expr) -> None:
    display_with_text("Функция:", sympy.Eq(Symbol('f'), f))

    x_domain = continuous_domain(f, x, S.Reals)
    y_domain = continuous_domain(f, y, S.Reals)
    stat_points = nonlinsolve([simplify(f.diff(x)), simplify(f.diff(y))], [x, y])
    found: bool = False
    zero_delta: bool = False
    for point in stat_points:
        x0, y0 = point
        if not (x0.is_real and x0 in x_domain and y0.is_real and y0 in y_domain):
            continue

        subs_dict = {x: x0, y: y0}
        delta = Delta(f, x, y).subs(subs_dict)
        if delta < 0:
            display_with_text(expression=f"\\text{{Стационарная точка }}{latex(point)}\\text{{ не является точкой экстремума}}")
            continue
        elif delta == 0:
            zero_delta = True
            display_with_text(expression=f"\\text{{В стационарной точке }}{point}\\text{{ det = 0, необходимо исслодовать функцию дальше}}")
            continue
        
        A = f.diff(x, 2).subs(subs_dict)
        B = f.diff(y, 2).subs(subs_dict)
        if A > 0 or B > 0:
            found = True
            display_with_text("minimum:", point)
        # aka A < 0 or B < 0
        elif A != 0 or B != 0:
            found = True
            display_with_text("maximum:", point)
    
    if not found:
        display_with_text("Точки экстремума не найдены" if zero_delta else "Точек экстремума нет")

x, y = symbols("x y", real=True)
f = x*x + x*y - 2*x - y
find_extr_points(f, x, y)

x, y = symbols("x y", positive=True, real=True)
f = x**3*y**2*(6 - x - y)
find_extr_points(f, x, y)

x, y = symbols("x y", real=True)
f = 8/x + x/y + y
find_extr_points(f, x, y)

f = (1 + x - y) / sympy.sqrt(1 + x*x + y*y)
find_extr_points(f, x, y)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### Задание 2.
С помощью метода Лагранжа найти условный экстремум функции
$z = x + 2y$ при условии $x^2 + y^2 = 5$.

In [84]:
x, y, lmbd = symbols("x y lambda")
restriction = x*x + y*y - 5
f = x + 2*y
L = f + lmbd*restriction
variables = [x, y, lmbd]
stat_points = nonlinsolve([L.diff(var) for var in variables], variables)
display_with_text(None, "\\text{{Стационарные точки относительно }} x, y, \lambda: ", *stat_points)

dx, dy, dx2, dy2 = symbols('dx dy dx^2 dy^2')
d2L = factor(L.diff(x,2)*dx2 + 2*L.diff(x,y)*dx*dy + L.diff(y,2)*dy2)
display_with_text("Выражение второго дифференциала формулы Лагранжа:", d2L)

f_symb = Symbol('f')
min_f = set()
max_f = set()
for point in stat_points:
    f_value = f.subs({var: value for var, value in zip(variables, point)})
    if point[-1] > 0:
        min_f.add(f_value)
    else:
        max_f.add(f_value)
    display_with_text(f"В точке {point}:", sympy.Eq(f_symb, f_value))

display_with_text("Точки максимума функции:", *max_f)
display_with_text("Точки минимума функции:", *min_f)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### Задание 3.
С помощью метода Лагранжа найти условный экстремум функции
$z=x^2+y^2$ при условии $\frac{x}{2}+\frac{y}{3}=1$.

In [85]:
x, y, lmbd = symbols("x y lambda")
f = x*x + y*y
restriction = x/2 + y/3 - 1
L = f + lmbd*restriction

variables = [x, y, lmbd]
stat_points = nonlinsolve([L.diff(var) for var in variables], variables)
display_with_text(None, "\\text{{Стационарные точки относительно }} x, y, \lambda: ", *stat_points)

dx, dy, dx2, dy2 = symbols('dx dy dx^2 dy^2')
d2L = factor(L.diff(x,2)*dx2 + 2*L.diff(x,y)*dx*dy + L.diff(y,2)*dy2)
display_with_text("Выражение второго дифференциала формулы Лагранжа:", d2L)
display_with_text(expression="\\text{{Выражение не зависит от коэффициента }}\lambda\\text{{ функции Лагранжа и всегда положительно}}")

f_symb = Symbol('f')
for point in stat_points:
    f_value = f.subs({var: value for var, value in zip(variables, point)})
    display_with_text(f"Точка минимума в точке {point}:", sympy.Eq(f_symb, f_value))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### Задание 4.
С помощью метода Лагранжа найти экстремум функции $u = x - 2y + 2z$ при условии $x^2 + y^2 + z^2 = 9$.

In [86]:
x, y, z, lmbd = symbols("x y z lambda")
f = x - 2*y + 2*z
restriction = x*x + y*y + z*z - 9
L = f + lmbd*restriction

variables = (x, y, z, lmbd)
stat_points = nonlinsolve([L.diff(var) for var in variables], variables)
display_with_text(None, "\\text{{Стационарные точки относительно }} x, y, z, \lambda: ", *stat_points)

dx, dy, dz, dx2, dy2, dz2 = symbols('dx dy dz dx^2 dy^2 dz^2')
d2L = 0
diffs = {x: (dx, dx2), y: (dy, dy2), z: (dz, dz2)}
for i, var in enumerate((x, y, z)):
    d2L += L.diff(var, 2)*diffs[var][1]
    for j in range(i + 1, len(variables) - 1):
        d2L += 2*L.diff(var, variables[j])*diffs[var][0]*diffs[variables[j]][0]
d2L = factor(d2L)
display_with_text("Выражение второго дифференциала формулы Лагранжа:", d2L)

lambda_vals = {stat_point[-1] for stat_point in stat_points}
display_with_text(None, "\\text{{Значения параметра }}\lambda:", *lambda_vals)
f_symb = Symbol('f')
min_f = set()
max_f = set()
for point in stat_points:
    f_value = f.subs({var: value for var, value in zip(variables, point)})
    if point[-1] > 0:
        min_f.add(f_value)
    else:
        max_f.add(f_value)
    display_with_text(f"В точке {point}:", sympy.Eq(f_symb, f_value))

display_with_text("Точки максимума функции:", *max_f)
display_with_text("Точки минимума функции:", *min_f)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### Индивидуальное задание
Найти точки экстремума $u = x^2 + y^2 + z^2 - xy + x - 2z$.

Вариант 50
$ f (x, y, z) = −x^2+ 5xy + 2xz −2x −5y^2+ 4y −5z^2−z $

In [87]:
from sympy.calculus.util import continuous_domain

def Delta(f: Expr, x: Expr, y: Expr, z: Expr):
    m: Matrix = Matrix((
        (f.diff(x, 2), f.diff(x, y), f.diff(x, z)),
        (f.diff(y, x), f.diff(y, 2), f.diff(y, z)),
        (f.diff(z, x), f.diff(z, y), f.diff(z, 2))
    ))
    m.simplify()

    return m.det()

def find_extr_points(f: Expr, x: Expr, y: Expr, z: Expr) -> None:
    display_with_text("Функция:", sympy.Eq(Symbol('f'), f))

    x_domain = continuous_domain(f, x, S.Reals)
    y_domain = continuous_domain(f, y, S.Reals)
    z_domain = continuous_domain(f, z, S.Reals)
    stat_points = nonlinsolve([simplify(f.diff(x)), simplify(f.diff(y)), simplify(f.diff(z))], [x, y, z])
    min_f = set()
    max_f = set()
    zero_delta: bool = False
    for point in stat_points:
        x0, y0, z0 = point
        if not (x0.is_real and y0.is_real and z0.is_real):
            continue
        if not (x0 in x_domain and y0 in y_domain and z0 in z_domain):
            pass
            # continue

        subs_dict = {x: x0, y: y0, z: z0}
        delta = Delta(f, x, y, z).subs(subs_dict)
        if delta < 0:
            display_with_text(expression=f"\\text{{Стационарная точка }}{latex(point)}\\text{{ не является точкой экстремума}}")
            continue
        elif delta == 0:
            zero_delta = True
            display_with_text(expression=f"\\text{{В стационарной точке }}{point}\\text{{ det = 0, необходимо исслодовать функцию дальше}}")
            continue
        
        A = f.diff(x, 2).subs(subs_dict)
        B = f.diff(y, 2).subs(subs_dict)
        C = f.diff(z, 2).subs(subs_dict)
        if A > 0 or B > 0 or C > 0:
            min_f.add(point)
        # aka A < 0 or B < 0 or C < 0
        elif A != 0 or B != 0 or C != 0:
            max_f.add(point)
    
    if max_f or min_f:
        if max_f:
            display_with_text("Точки максимума функции:", *max_f)
        if min_f:
            display_with_text("Точки минимума функции:", *min_f)
    else:
        display_with_text("Точки экстремума не найдены" if zero_delta else "Точек экстремума нет")

x, y, z = symbols("x y z", real=True)
u = x*x + y*y + z*z - x*y + x - 2*z
find_extr_points(u, x, y, z)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>